In [32]:
import json
import matplotlib.pyplot as plt
import numpy as np
from scipy.linalg import norm, solve
from scipy.optimize import Bounds, minimize

%matplotlib inline
plt.rc('text', usetex=True)
plt.rcParams.update({'font.size': 15})

def list2col(lst):
    return np.array(lst[::-1]).reshape((len(lst), 1))

In [33]:
data_file = open('colombia.json',)
json_data = json.load(data_file)
var = 'Y'

# ARMA LS

For data $\{I_{-2h},\ldots,I_{-1},I_{0},\ldots,I_t,I_{t+1}\}$

In [36]:
# time_series : 1D - np array (t+2,)
# h1          : int

def estimate_r(time_series, h1):
    if 2*h1 >= len(time_series) - 1:
        return None, None
    
    T = len(time_series) - 2*h1 - 2
    x = lambda t: list2col(time_series[(t + h1):(t + 2*h1 + 1)])
    v = lambda t: x(t - h1)
    
    V = np.zeros((h1 + 1, h1 + 1))
    Z = np.zeros((h1 + 1, h1 + 1))
    for s in range(T + 1):
        V += np.matmul(v(s), np.transpose(x(s+1)))
        Z += np.matmul(x(s), np.transpose(v(s)))
    
    A = np.matmul(np.transpose(V), np.linalg.inv(Z))
    
    betas = A[0, :]
    R = betas.sum()
    return R, A

# Estimate $R$ - AR Constrained LS

In [57]:
def estimate_r_opt(time_series, h, show=False):
    if 2*h >= len(time_series) - 1:
        return None
    
    # "Current" t in time series
    T = len(time_series) - 2*h - 2
    x = lambda t: list2col(time_series[(t + h):(t + 2*h + 1)])
    
    A = []
    for s in range(h + 1):
        A.append(x(T-s))
    A = np.hstack(tuple(A))
    
    def of(R):
        aux = np.matmul(A, R.reshape((R.size, 1)))
        return np.power(norm(x(T+1) - aux), 2)
    
    init = np.zeros(h+1)
    bnds = Bounds([0.0001 for s in range(h+1)], [np.inf for s in range(h+1)])
    sol = minimize(of, init, bounds=bnds)
    if show:
        print(sol)
    return sol.x.sum()

## Dynamics of $R$ w.r.t $h$ and $t^*$

In [12]:
def analyze_r(time_series, h=14, h0=2, title=''):
    Rsh = []
    Rst = []
    index = -1
    delta_h = 0
    first = False
    second = False
    while True:
        if len(time_series[:index]) < 2*h0 + 2:
            first = True
        if len(time_series) < 2*(h0 + delta_h) + 2:
            second = True
        if first or second:
            break
        if not first:
            Rst.append(estimate_r_opt(time_series[:index], h))
            index -= 1
        if not second:
            Rsh.append(estimate_r_opt(time_series, h0 + delta_h))
            delta_h += 1
        
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=[12, 5])
    ax1.plot(Rsh, 'k')
    ax1.set_xlabel('$h$')
    ax1.set_ylabel('$R_0$')
    
    ax2.plot(Rst, 'k')
    ax2.set_xlabel('$t^*$')
    ax2.set_ylabel('$R_0$')
    fig.suptitle(title)
    plt.show()
    
    return Rsh, Rst

## $R_0$

In [50]:
m = 90
h = 40

In [61]:
# colombia
print(estimate_r_opt(json_data['co'][var][-m:], h, show=True))

      fun: 169121426.4572094
 hess_inv: <41x41 LbfgsInvHessProduct with dtype=float64>
      jac: array([2.10888213e+08, 3.73699003e+08, 4.80582809e+08, 5.35408446e+08,
       5.69143501e+08, 5.94247794e+08, 6.24125451e+08, 6.42108387e+08,
       6.58649209e+08, 6.58043247e+08, 6.50034526e+08, 6.42265877e+08,
       6.39607617e+08, 6.41147590e+08, 6.63591471e+08, 6.84366190e+08,
       7.13179353e+08, 7.01829144e+08, 6.70051563e+08, 6.42410016e+08,
       6.32274878e+08, 6.28328371e+08, 6.19032183e+08, 5.95828885e+08,
       5.60506532e+08, 5.15756455e+08, 4.95965037e+08, 5.01640189e+08,
       5.16031030e+08, 5.25346693e+08, 5.27421713e+08, 5.15817213e+08,
       5.17173404e+08, 5.17537344e+08, 5.24972066e+08, 5.43376848e+08,
       5.57820213e+08, 5.61955330e+08, 5.52305913e+08, 5.35407156e+08,
       5.19950292e+08])
  message: b'CONVERGENCE: REL_REDUCTION_OF_F_<=_FACTR*EPSMCH'
     nfev: 4284
      nit: 66
   status: 0
  success: True
        x: array([4.83975193e-01, 3.19581970e-0

In [59]:
# bogotá
print(estimate_r_opt(json_data['co_11'][var][-m:], h))

1.0359824647046056


In [60]:
# medellín
print(estimate_r_opt(json_data['co_05001'][var][-m:], h))

1.296798445816166
